In [64]:
#Importamos librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### ***Cargamos los datasets***

In [65]:
df_train = pd.read_parquet("dataset/train.parquet")
df_test = pd.read_parquet("dataset/test.parquet")

### ***Análisis exploratorio***

In [45]:
#Tipos de datos en columnas
df_train.dtypes

id                           int64
url                         object
region                      object
region_url                  object
price                        int64
type                        object
sqfeet                       int64
beds                         int64
baths                      float64
cats_allowed                 int64
dogs_allowed                 int64
smoking_allowed              int64
wheelchair_access            int64
electric_vehicle_charge      int64
comes_furnished              int64
laundry_options             object
parking_options             object
image_url                   object
description                 object
lat                        float64
long                       float64
state                       object
dtype: object

In [46]:
#Cantidad de filas y columnas
df_train.shape

(346479, 22)

In [47]:
#Cantidad de filas y columnas
df_test.shape

(38498, 21)

In [48]:
#Todas las columnas del dataframe
df_train.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'type', 'sqfeet', 'beds',
       'baths', 'cats_allowed', 'dogs_allowed', 'smoking_allowed',
       'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished',
       'laundry_options', 'parking_options', 'image_url', 'description', 'lat',
       'long', 'state'],
      dtype='object')

In [49]:
#Valores faltantes en columnas
df_train.isna().sum()

id                              0
url                             0
region                          0
region_url                      0
price                           0
type                            0
sqfeet                          0
beds                            0
baths                           0
cats_allowed                    0
dogs_allowed                    0
smoking_allowed                 0
wheelchair_access               0
electric_vehicle_charge         0
comes_furnished                 0
laundry_options             71171
parking_options            126682
image_url                       0
description                     2
lat                          1722
long                         1722
state                           0
dtype: int64

In [50]:
#Detecto duplicados en columna "image_url"
#Podria significar que hay varios anuncios publicitando la misma casa
df_train.duplicated(subset="image_url").value_counts()

True     178233
False    168246
dtype: int64

In [51]:
df_test.duplicated(subset="image_url").value_counts()

False    30546
True      7952
dtype: int64

In [52]:
#Detecto que los valores en latitud y longitud son incorrectos
df_train[["lat", "long"]].describe()

,lat,long
count,344757.000000,344757.000000
mean,37.234363,-92.705415
std,5.550956,16.551071
min,-43.533300,-163.894000
25%,33.453100,-100.784000
50%,37.650100,-87.710800
75%,41.137900,-81.174600
max,102.036000,172.633000


### ***Feature Engineering***

En primer lugar voy a descartar las columnas que no son determinantes en el precio del inmueble

In [53]:
df_train.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'type', 'sqfeet', 'beds',
       'baths', 'cats_allowed', 'dogs_allowed', 'smoking_allowed',
       'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished',
       'laundry_options', 'parking_options', 'image_url', 'description', 'lat',
       'long', 'state'],
      dtype='object')

In [66]:
df_train.drop(columns=['id', 'url', 'region_url', 'image_url', 'description'], inplace=True)

También descarto las columnas de "latitud" y "longitud", ya que poseen valores incorrectos y tengo otros features como "state" y "region" que proveen información de la ubicación del inmueble.

In [67]:
df_train.drop(columns=["lat", "long"], inplace=True)

Así nos quedó el dataframe:

In [56]:
df_train.columns

Index(['region', 'price', 'type', 'sqfeet', 'beds', 'baths', 'cats_allowed',
       'dogs_allowed', 'smoking_allowed', 'wheelchair_access',
       'electric_vehicle_charge', 'comes_furnished', 'laundry_options',
       'parking_options', 'state'],
      dtype='object')

Para que el modelo de ML pueda comprender toda la información, vamos a convertir las columnas string en un valor numérico

In [68]:
#Convertimos cada valor string en un número
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

columnas = ['region', 'type', 'state', 'laundry_options', 'parking_options']
for columna in columnas:
    df_train[columna] = label_encoder.fit_transform(df_train[columna])

In [69]:
#Cambiamos el tipo de dato de las columnas
columnas = {'region': "int64", 'type': "int64", 'state': "int64", 
            'laundry_options': "int64", 'parking_options': "int64", 'baths': "int64"}
df_train = df_train.astype(columnas)

In [70]:
df_train.dtypes

region                     int64
price                      int64
type                       int64
sqfeet                     int64
beds                       int64
baths                      int64
cats_allowed               int64
dogs_allowed               int64
smoking_allowed            int64
wheelchair_access          int64
electric_vehicle_charge    int64
comes_furnished            int64
laundry_options            int64
parking_options            int64
state                      int64
dtype: object

In [68]:
df_train.head(2)

,region,price,type,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,state
0,33,1350,6,1200,2,2,1,1,1,0,0,0,4,2,13
1,64,1115,0,694,1,1,1,1,1,0,0,0,4,1,5


Creamos la columna "category_price" que nos indica si el inmueble tiene precio "high", "medium" o "low"

In [71]:
col = "price"
conditions = [df_train[col] >= 2000, (df_train[col] < 1999) & (df_train[col] > 999), df_train[col] <= 999]
choices = ["high", "medium", "low"]

df_train["category_price"] = np.select(conditions, choices, default=np.nan)

Creamos la columna "is_low" para indicar si el inmueble tiene precio "low" o no.

In [72]:
#Si toma el valor 1, significa que el valor es menor a 999 dolares, osea es "low"
df_train["is_low"] = np.where(df_train["price"] <= 999, 1, 0)

Así nos quedo el dataframe:

In [86]:
df_train.head()

,region,price,type,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,state,category_price,is_low
0,33,1350,6,1200,2,2,1,1,1,0,0,0,4,2,13,medium,0
1,64,1115,0,694,1,1,1,1,1,0,0,0,4,1,5,medium,0
2,236,1129,0,900,2,2,0,0,1,0,0,0,3,4,45,medium,0
3,269,1580,6,1469,3,2,1,1,1,0,0,0,4,7,3,medium,0
4,230,995,0,700,1,1,1,1,1,0,0,0,4,1,42,low,1


### **`Machine Learning: Árbol de Decisión`**

Elegí este modelo porque la problemática consistía en **`clasificar si un inmueble posee precio "low" o no`**. Por lo tanto, dentro de las diferentes variables de modelos de clasificación, el árbol de decisión posee las cualidades que según yo, se adaptan mejor a los datos que se deben trabajar <br>

Esas cualidades son:
- **Fácilmente comprensible:** los árboles de decisión son fáciles de entender e interpretar, ya que proporcionan una forma de visualizar los algoritmos.

- **Requisito bajo o nulo para el preprocesamiento de datos:** a diferencia de otros algoritmos, los árboles de decisión toman menos tiempo para el modelado, ya que necesitan poco análisis, codificación y variables ficticias porque para cada punto de datos, habrá un conjunto completo.

- **Versatilidad de los datos:** podemos estandarizar los datos aunque no los poseamos. Podemos incluir datos categóricos y numéricos, ya que funciona bien con ambos.

In [73]:
#Importamos las librerias

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [74]:
#Instanciamos el modelo con una profundidad de 18
tree = DecisionTreeClassifier(max_depth = 18)

#Seleccionamos los features y el target
X = df_train[['region', 'sqfeet', 'beds', 'baths', 'cats_allowed',
       'dogs_allowed', 'smoking_allowed', 'wheelchair_access',
       'electric_vehicle_charge', 'comes_furnished', 'state', 'laundry_options', 'parking_options']]
y = df_train["is_low"]

#Dividimos el dataframe para poder testearlo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [75]:
#Entrenamos el modelo
tree.fit(X_train, y_train)

#Hacemos la predicción
y_pred = tree.predict(X_test)

Verificamos el accuracy del modelo

In [76]:
print("Accuracy en el set de entrenamiento", tree.score(X_train, y_train))
print("Accuracy en el set de testeo", tree.score(X_test, y_test))

Accuracy en el set de entrenamiento 0.9426103448986745
Accuracy en el set de testeo 0.9047083044716385


Obtuvimos un gran desempeño del modelo y se puede decir que es capaz de predecir efectivamente. El accuracy en el set de entrenamiento nos indica que tanto el modelo se estudio los datos de memoria y su capacidad de generalizar (overfitting-underfitting). El accuracy en el set de testeo nos indica como respondió el modelo frente a nueva información.

#### `Validación Cruzada`

Aplico "cross_validate" para obtener una estimación menos sesgada y más realista del funcionamiento del modelo.

In [77]:
from sklearn.model_selection import cross_validate

In [78]:
tree_scores = cross_validate(tree, X, y, cv=5, scoring = ['accuracy','recall'])

In [79]:
print(tree_scores)

{'fit_time': array([1.06355786, 1.05167055, 1.05037379, 0.97621989, 0.99222469]), 'score_time': array([0.04153013, 0.04103994, 0.04100966, 0.04001045, 0.0410099 ]), 'test_accuracy': array([0.90529035, 0.90243304, 0.9047997 , 0.90472755, 0.90422108]), 'test_recall': array([0.88325916, 0.88160966, 0.88095609, 0.88484641, 0.88568316])}


### **`Machine Learning: Random Forest`**

Anteriormente vimos que el árbol de decisión obtuvo un buen desempeño, pero podemos mejorarlo utilizando RandomForest, que es un conjunto de árboles de decisión <br>

Aplicar RandomForest posee las siguientes ventajas:
- **Entrenamiento aleatorio:** por cada árbol creado, se selecciona un set de entrenamiento distinto, lo que reduce el sesgo del modelo.

- **Features aleatorios:** por cada árbol creado, el modelo selecciona features aleatorios al dividir los nodos. Lo que permite detectar los features más importantes y más determinantes.

In [80]:
#Importamos el modelo
from sklearn.ensemble import RandomForestClassifier

In [81]:
#Instanciamos el modelo con cantidad de 200 árboles.
clf = RandomForestClassifier(n_estimators=200)

#Seleccionamos los features y el target
X = df_train[['region', 'type', 'sqfeet', 'beds', 'baths', 'cats_allowed',
       'dogs_allowed', 'smoking_allowed', 'wheelchair_access',
       'electric_vehicle_charge', 'comes_furnished', 'state', 'laundry_options', 'parking_options']]
y = df_train["is_low"]

#Dividimos el dataframe para testearlo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [82]:
#Entrenamos el modelo
clf.fit(X_train,y_train)

#Hacemos la predicción
y_pred = clf.predict(X_test)

Verificamos el accuracy del modelo

In [83]:
print("Accuracy en el set de entrenamiento", clf.score(X_train, y_train))
print("Accuracy en el set de testeo", clf.score(X_test, y_test))

Accuracy en el set de entrenamiento 0.9867153194384316
Accuracy en el set de testeo 0.9358885553759717


#### **`Features con mayor importancia`**

In [84]:
feature_imp = pd.Series(clf.feature_importances_,index=clf.feature_names_in_).sort_values(ascending=False)
feature_imp

sqfeet                     0.252789
state                      0.221579
region                     0.194270
laundry_options            0.092733
parking_options            0.061191
baths                      0.049108
beds                       0.037176
type                       0.027427
smoking_allowed            0.017375
dogs_allowed               0.011915
cats_allowed               0.011607
comes_furnished            0.010637
wheelchair_access          0.009685
electric_vehicle_charge    0.002507
dtype: float64

Detectamos que **`"sqfeet"`** (pies cuadrados del inmueble), **`"state"`** y **`"region"`** (estado y región donde se ubica el inmueble) son los features más determinantes a la hora de predecir el valor de la propiedad

#### Creacion de un nuevo RandomForest con los features mas importantes

In [87]:
#Instanciamos el modelo
clf2 = RandomForestClassifier(n_estimators=200)

#Seleccionamos features y target
X = df_train[['region', 'sqfeet', 'state']]
y = df_train["is_low"]

#Separamos el dataframe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [88]:
#Entrenamos el modelo
clf2.fit(X_train,y_train)

#Hacemos la predicción
y_pred = clf2.predict(X_test)

In [89]:
print("Accuracy en el set de entrenamiento", clf2.score(X_train, y_train))
print("Accuracy en el set de testeo", clf2.score(X_test, y_test))

Accuracy en el set de entrenamiento 0.9507617457274208
Accuracy en el set de testeo 0.9101150619564381


Obtuvimos un resultado muy similar y **`pasamos de 14 features a solamente 3!`**. Ademas reducimos notablemente los tiempos del modelo en entrenarse y predecir los resultados. <br>

####  Aplicamos la misma transformación a "df_test"

In [90]:
#Eliminamos columnas
df_test.drop(columns=['id', 'url', 'region_url', 'image_url', 'description'], inplace=True)
df_test.drop(columns=["lat", "long"], inplace=True)

In [91]:
#Convertimos cada valor string en un número
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

columnas = ['region', 'type', 'state', 'laundry_options', 'parking_options']
for columna in columnas:
    df_test[columna] = label_encoder.fit_transform(df_test[columna])

In [92]:
#Cambiamos el tipo de dato de las columnas
columnas = {'region': "int64", 'type': "int64", 'state': "int64", 
            'laundry_options': "int64", 'parking_options': "int64", 'baths': "int64"}
df_test = df_test.astype(columnas)

#### Testeamos el modelo ya entrenado con df_test

In [93]:
#Seleccionamos los features del X_test
X = df_test[['region', 'type', 'sqfeet', 'beds', 'baths', 'cats_allowed',
       'dogs_allowed', 'smoking_allowed', 'wheelchair_access',
       'electric_vehicle_charge', 'comes_furnished', 'state', 'laundry_options', 'parking_options']]

In [94]:
#Predecimos
pred = clf.predict(X)

In [95]:
#Convertimos la predicción a un dataframe
prediccion = pd.DataFrame(data=pred, columns=["pred"])

In [96]:
#Visualizamos el dataframe
prediccion

,pred
0,0
1,1
2,0
3,0
4,0
...,...
38493,0
38494,1
38495,0
38496,0


Finalmente, exportamos las predicciones a formato csv

In [97]:
prediccion.to_csv("prediccion.csv", index=False)